# Synthesis
The main method of synthesis implemented here is "Gradient Matching" derived from [this](https://openreview.net/pdf?id=mSAKhLYLSsl) paper and code from [this](https://github.com/dm-medvedev/EfficientDistillation) repository. In a nutshell, the synthesized imageset is iteratively developed by taking sample minibatches of the training data, in this case Fashin MNIST, and computing the loss by applying a deep neural network. The associated batch of synthetic data with the samples is then updated by using gradient descent to match the change in loss with the training sample's. The parameters used on the deep neural network is updated to minimize loss after each iteration in which the synthetic data is updated. The end result is a fully synthesized data set. 

## Import
The primary resource used is PyTorch's 'torch

In [ ]:
import os
import time
import copy
import argparse
import numpy as np
import torch
import torch.nn as nn
from torchvision.utils import save_image
from utils import get_loops, get_dataset, get_network, get_eval_pool, evaluate_synset, get_daparam, match_loss, get_time, TensorDataset, epoch, DiffAugment, ParamDiffAug